<a href="https://colab.research.google.com/github/deanjetblue/blue_grabber/blob/main/blue_grabber.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Mounting Google Drive Account
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive

#Installing bs4 OCR tool
!pip install beautifulsoup4
!apt-get install ocrmypdf
!apt install ghostscript python3-tk
# Importing Packages
import pandas as pd
import os
import time
from os import path
from glob import glob
import requests as req
from bs4 import BeautifulSoup
import subprocess


Mounted at /gdrive
/gdrive
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fonts-droid-fallback fonts-noto-mono fonts-urw-base35 ghostscript icc-profiles-free libgs9
  libgs9-common libidn12 libijs-0.35 libimagequant0 libjbig2dec0 libqpdf28 libraqm0 mailcap
  mime-support pngquant poppler-data python3-bs4 python3-chardet python3-coloredlogs
  python3-html5lib python3-humanfriendly python3-img2pdf python3-lxml python3-olefile
  python3-packaging python3-pdfminer python3-pikepdf python3-pil python3-pluggy python3-renderpm
  python3-reportlab python3-reportlab-accel python3-soupsieve python3-tqdm python3-webencodings
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd unpaper
Suggested packages:
  fonts-noto fonts-freefont-otf | fonts-freefont-ttf fonts-texgyre ghostscript-x ocrmypdf-doc
  python-watchdog img2pdf poppler-utils fonts-japanese-mincho | fonts-ipafont-mincho
  fonts-japanes

In [3]:
# Load the CSV file that includes all ICAO Airports
icaoFilePath = r'/gdrive/My Drive/JetBlueProjects/8260s/Current ICAOs/2024-12-20 4_17pm.csv'
folderPath = '/gdrive/My Drive/JetBlueProjects/8260s/Airports/'
df = pd.read_csv(icaoFilePath)
icaoList = df.iloc[0:, 0].tolist()

In [4]:
def blue_grabber_8260 (icaos, folderName):
    for icao in icaos:
      countSucc = 0
      countErr = 0
      print('*****************'+icao+'*****************')
      # Creates Path Directory for PDFs to be stored, if not already created ({folderPath}/SampleFolder/MCO/)
      if not os.path.exists(folderPath + folderName + icao):
        file_path = os.makedirs(folderPath + folderName + icao)
      # Saves the URL to the NDBR Webpage
      url1 = (
                    'https://www.faa.gov/air_traffic/flight_info/aeronav/procedures/application/?event=procedure.Results&tab=ndbr&nasrId='
                    + icao
                    + '#searchResultsTop')
      url2 = (
                    'https://www.faa.gov/air_traffic/flight_info/aeronav/procedures/application/?event=procedure.Results&tab=ndbr&nasrId='
                    + icao
                    + '&startRow=51#searchResultsTop')
      url3 = (
                    'https://www.faa.gov/air_traffic/flight_info/aeronav/procedures/application/?event=procedure.Results&tab=ndbr&nasrId='
                    + icao
                    + '&startRow=101#searchResultsTop')
      url4 = (
                    'https://www.faa.gov/air_traffic/flight_info/aeronav/procedures/application/?event=procedure.Results&tab=ndbr&nasrId='
                    + icao
                    + '&startRow=151#searchResultsTop')

      for url in [url1, url2, url3, url4]:
        # Sends GET request to the URL
          response = req.get(url)
          # If Webpage Sucess enter for loop to find table
          if response.status_code == 200:
              #Parsing the html file to find the table in the NDBR webpage and store the content
              soup = BeautifulSoup(response.text, 'html.parser')
              links = soup.select('table tbody tr td:first-child a')
              #Looping through the table found on NDBR webpage
              if links != []:
                for link in links:
                        #This page in the parent directory of all links
                        indexCfm = 'https://www.faa.gov/air_traffic/flight_info/aeronav/procedures/application/'
                        linkText = link['href']
                        #Appending the parent directory to the child directory and sending GET request
                        linkURL = req.get(indexCfm+linkText[9:])
                        soup2 = BeautifulSoup(linkURL.text, 'html.parser')
                        links2 = soup2.select('table tbody tr td:first-child a')
                        #If there is a table on webpage enter for loop
                        if links2 != []:
                            for link2 in links2:
                                print(links2[0]['href'])
                                print(link)
                                pdfLink = 'https://www.faa.gov'+link2['href']
                                print(pdfLink)
                                linkClick = req.get('https://www.faa.gov'+link2['href'])
                                linkClickName = link2['href'].split('/')[-1].replace('.pdf','')
                                print(linkClickName)
                                if linkClick.status_code == 200:
                                    with open(folderPath + folderName + icao+ '/' + linkClickName + '.pdf',  'wb') as file:
                                        # Save this as CSV File
                                        file.write(linkClick.content)
                                    countSucc = countSucc + 1
                                    print('File downloaded successfully' + ' Sucess Downloads Count is: ' + str(countSucc))
                                else:
                                    countErr = countErr + 1
                                    print('Error: File failed to download ' + 'Failed Download Count is: ' + str(countErr))
                        else:
                            print(indexCfm+linkText[9:])
                            countErr = countErr + 1
                            print('Error: File failed to download ' + 'Failed Download Count is: ' + str(countErr))
    print('\n\n\n\n\n\nFinal Counts: \nSucessful: ' + str(countSucc) + '\nFailed: ' + str(countErr) )
#blue_ocr(folderPath + folderName + icao)


In [6]:
blue_grabber_8260(['SAV'],'zachAttack')

*****************SAV*****************
/aero_docs/acifp/NDBR/5D8C99DD0D874B0D884DDFC2DCABE4E5-SAV-NDBR/GA_SAV_IL01_AMDT%208B.pdf
<a href="index.cfm?event=directory&amp;directory=5D8C99DD0D874B0D884DDFC2DCABE4E5-SAV-NDBR&amp;type=ndbr&amp;nasrId=SAV">ILS OR LOC RWY 1 AMDT 8C</a>
https://www.faa.gov/aero_docs/acifp/NDBR/5D8C99DD0D874B0D884DDFC2DCABE4E5-SAV-NDBR/GA_SAV_IL01_AMDT%208B.pdf
GA_SAV_IL01_AMDT%208B
File downloaded successfully Sucess Downloads Count is: 1
/aero_docs/acifp/NDBR/5D8C99DD0D874B0D884DDFC2DCABE4E5-SAV-NDBR/GA_SAV_IL01_AMDT%208B.pdf
<a href="index.cfm?event=directory&amp;directory=5D8C99DD0D874B0D884DDFC2DCABE4E5-SAV-NDBR&amp;type=ndbr&amp;nasrId=SAV">ILS OR LOC RWY 1 AMDT 8C</a>
https://www.faa.gov/aero_docs/acifp/NDBR/5D8C99DD0D874B0D884DDFC2DCABE4E5-SAV-NDBR/P-NOTAM_GA_SAV_IL01_AMDT%208C.pdf
P-NOTAM_GA_SAV_IL01_AMDT%208C
File downloaded successfully Sucess Downloads Count is: 2
/aero_docs/acifp/NDBR/2018010832110601001-SAV-NDBR/GA_SAV_ILS%20OR%20LOC%20RWY%2010,%20A

In [ ]:
def blue_ocr(directory_path):
    directory_path = '/gdrive/My Drive/JetBlueProjects/8260s/Airports' + '/' + directory_path
    pdfs = os.listdir(directory_path)
    pdfs = glob(path.join(directory_path, '*pdf'))
    if not os.path.exists(directory_path + '-Clean'):
      os.mkdir(directory_path + '-Clean')
    pdfsClean = [path.replace(directory_path, directory_path + '-Clean') for path in pdfs]




    for pdf, pdfClean in zip(pdfs, pdfsClean):
      gs = ["gs", "-dPDFA=1", "-dBATCH", "-r300x300", "-dNOPAUSE", "-sDEVICE=pdfwrite", f"-sOutputFile={pdfClean}", pdf]
      subprocess.run(gs, check=True, text=True, capture_output=True)
      print('1')

    for pdfClean in pdfsClean:
        ocr = [
          "ocrmypdf",
          "--tesseract-oem", "1",
          "--tesseract-config", "-c tessedit_char_whitelist=0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz/",
          pdfClean,
          pdfClean,
          "--force-ocr"
            ]
        subprocess.run(ocr, check=True, text=True, capture_output=True)



In [ ]:
#blue_ocr('sampleMCO')